In [4]:
#importig scrapy for scraping data
import scrapy
from scrapy.item import Item, Field
from scrapy import Spider


class StackItem(Item):
    title = Field()
    url = Field()
    image = Field()
    description = Field()

In [5]:
class StackSpider(Spider):
    name = "stack"
    allowed_domains = ["news.ycombinator.com"]
    start_urls = [
        "news.ycombinator.com",
    ]
    def parse(self, response):
        items = Selector(response).xpath('//[@class="athing"]/tr')

        for item in items:
            item = StackItem()
            item['title'] = item.xpath(
                'a[@class="title"]/text()').extract()[0]
            item['url'] = item.xpath(
                'a[@class="url"]/@td').extract()[0]
            item['image'] = item.xpath(
                'a[@class="img"]/@tr').extract()[0]
            item['description'] = item.xpath(
                'a[@class="body"]/@td').extract()[0]
            yield item

In [6]:
# create the database specify the pipeline and add the database settings

ITEM_PIPELINES = ['stack.pipelines.MongoDBPipeline', ]

MONGODB_SERVER = "localhost"
MONGODB_PORT = 27017
MONGODB_DB = "stackoverflow"
MONGODB_COLLECTION = "questions"

In [ ]:
#Storing the data in mongodb

import pymongo

from scrapy.conf import settings
from scrapy.exceptions import DropItem
from scrapy import log


class MongoDBPipeline(object):

    def __init__(self):
        connection = pymongo.MongoClient(
            settings['MONGODB_SERVER'],
            settings['MONGODB_PORT']
        )
        db = connection[settings['MONGODB_DB']]
        self.collection = db[settings['MONGODB_COLLECTION']]

    def process_item(self, item, spider):
        valid = True
        for data in item:
            if not data:
                valid = False
                raise DropItem("Missing {0}!".format(data))
        if valid:
            self.collection.insert(dict(item))
            log.msg("Question added to MongoDB database!",
                    level=log.DEBUG, spider=spider)
        return item